# Data Preparation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None


#### Reading all data

In [3]:
orders = pd.read_csv('orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})


order_products_train = pd.read_csv('order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_prior = pd.read_csv('order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

product_features = pd.read_pickle('product_features.pkl')

user_features = pd.read_pickle('user_features.pkl')

user_product_features = pd.read_pickle('user_product_features.pkl')

products = pd.read_csv('products.csv')

aisles = pd.read_csv('aisles.csv')

departments = pd.read_csv('departments.csv')

#### merging train order data with orders

In [4]:
orders.head(1)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN


In [5]:
order_products_train.head(1)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1


In [6]:
train_orders = orders.merge(order_products_train, on = 'order_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


removing unnecessary columns from train_orders

In [7]:
train_orders.drop(['eval_set', 'add_to_cart_order', 'order_id'], axis = 1, inplace = True)

unique user_ids in train data

In [8]:
train_users = train_orders.user_id.unique()
train_users[:10]

array([ 1,  2,  5,  7,  8,  9, 10, 13, 14, 17])

keeping only train_users in the data

In [9]:
user_product_features.shape

(13307953, 11)

In [10]:
user_product_features.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.600000,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555555,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,17.600000,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666666,10,1.0,0.0,0.0


In [11]:
df = user_product_features[user_product_features.user_id.isin(train_users)]
df.head(1)

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.9,1.4,17.6,10,1.0,1.0,1.0


In [12]:
train_orders.head(1)

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered
0,1,11,4,8,14.0,196,1


In [13]:
df = df.merge(train_orders, on = ['user_id', 'product_id'], how = 'outer')
df.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,17.600000,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,19.555555,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1,12427,10.0,9.0,0.900000,3.300000,17.600000,10.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1,13032,3.0,2.0,0.666667,6.333333,21.666666,10.0,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


for order_number, order_dow, order_hour_of_day, days_since_prior_order, impute null values with mean values grouped by users as these products will also be potential candidate for order.

In [14]:
df['order_number'] = df['order_number'].fillna(df.groupby('user_id')['order_number'].transform('mean'))
df['order_dow'] = df['order_dow'].fillna(df.groupby('user_id')['order_dow'].transform('mean'))
df['order_hour_of_day'] = df['order_hour_of_day'].fillna(df.groupby('user_id')['order_hour_of_day'].transform('mean'))
df['days_since_prior_order'] = df['days_since_prior_order'].fillna(df.groupby('user_id')['days_since_prior_order'].transform('mean'))


Removing those products which were bought the first time in last order by a user

In [15]:
df.reordered.value_counts()

reordered
1.0    828824
0.0    555793
Name: count, dtype: int64

In [16]:
df.shape

(9030454, 16)

In [17]:
df.reordered.isnull().sum()

np.int64(7645837)

In [18]:
df = df[df.reordered != 0]

In [19]:
df.shape

(8474661, 16)

Now imputing 0 in reordered as they were not reordered by user in his/her last order.

In [20]:
df.reordered.fillna(0, inplace = True)

df.isnull().sum()

C:\Users\abhis\AppData\Local\Temp\ipykernel_53444\3168525350.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.reordered.fillna(0, inplace = True)


user_id                            0
product_id                         0
total_product_orders_by_user       0
total_product_reorders_by_user     0
user_product_reorder_percentage    0
avg_add_to_cart_by_user            0
avg_days_since_last_bought         0
last_ordered_in                    0
is_reorder_3                       0
is_reorder_2                       0
is_reorder_1                       0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
reordered                          0
dtype: int64

In [21]:
df.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,17.600000,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,19.555555,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,0.0,11.0,4.0,8.0,14.0,0.0
3,1,12427,10.0,9.0,0.900000,3.300000,17.600000,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,0.0
4,1,13032,3.0,2.0,0.666667,6.333333,21.666666,10.0,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


#### Merging product and user features

In [22]:
product_features.shape

(49677, 35)

In [23]:
user_features.shape

(206209, 20)

In [24]:
product_features.head(2)

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136,0.613391,716,716,276,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.385475,...,7.692492,2887550,1657973,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12,0.133333,78,78,8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.102564,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0


In [25]:
user_features.head(1)

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_proportion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41,0.694915,5.9,0.705833,6.0,6.0,9.0,0.666667,1.0,0.666667


In [26]:
print(df.shape)
print(product_features.shape)
print(user_features.shape)


(8474661, 16)
(49677, 35)
(206209, 20)


In [ ]:
# Randomly sample 10% of the rows from the DataFrame
df_sampled = df.sample(frac=0.1, random_state=42)

# Display the first few rows of the sampled DataFrame
print(df_sampled.head())


         user_id  product_id  total_product_orders_by_user  \
5848284   133414        5428                          19.0   
4727585   107612       49099                           1.0   
3044692    69469        8021                           1.0   
8354143   190574        6377                           1.0   
8620182   196693       17758                           3.0   

         total_product_reorders_by_user  user_product_reorder_percentage  \
5848284                            18.0                         0.947368   
4727585                             0.0                         0.000000   
3044692                             0.0                         0.000000   
8354143                             0.0                         0.000000   
8620182                             2.0                         0.666667   

         avg_add_to_cart_by_user  avg_days_since_last_bought  last_ordered_in  \
5848284                 1.947368                    5.736842             23.0   
4727585 

In [28]:
# Check the data types of the relevant columns
print(df['product_id'].dtype)
print(df['user_id'].dtype)
print(product_features['product_id'].dtype)
print(user_features['user_id'].dtype)


int32
int64
uint16
int64


In [29]:
df_sampled = df_sampled.merge(product_features, on = 'product_id', how = 'left')
df_sampled = df_sampled.merge(user_features, on = 'user_id', how = 'left')
df_sampled.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_proportion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,133414,5428,19.0,18.0,0.947368,1.947368,5.736842,23.0,1.0,1.0,...,101,0.604790,7.260870,0.642670,12.0,7.0,6.0,0.500000,0.857143,0.666667
1,107612,49099,1.0,0.0,0.000000,13.000000,16.000000,3.0,0.0,0.0,...,53,0.304598,19.333333,0.306129,18.0,18.0,16.0,0.500000,0.277778,0.437500
2,69469,8021,1.0,0.0,0.000000,1.000000,4.000000,18.0,0.0,0.0,...,93,0.550296,6.035714,0.481418,20.0,3.0,19.0,0.800000,0.333333,0.736842
3,190574,6377,1.0,0.0,0.000000,2.000000,17.000000,11.0,0.0,0.0,...,76,0.410811,5.606061,0.465079,6.0,5.0,1.0,0.333333,0.800000,1.000000
4,196693,17758,3.0,2.0,0.666667,9.333333,24.666666,13.0,0.0,0.0,...,223,0.563131,24.750000,0.573914,22.0,18.0,28.0,0.636364,0.500000,0.821429


The dataframe has null values because the product was never bought earlier by a user

In [ ]:
df_sampled.shape

(8474661, 16)

In [ ]:
df_sampled.isnull().sum().sort_values(ascending = False)

user_id                            0
product_id                         0
total_product_orders_by_user       0
total_product_reorders_by_user     0
user_product_reorder_percentage    0
avg_add_to_cart_by_user            0
avg_days_since_last_bought         0
last_ordered_in                    0
is_reorder_3                       0
is_reorder_2                       0
is_reorder_1                       0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
reordered                          0
dtype: int64

In [35]:
df_sampled.to_pickle('Finaldata.pkl')

In [36]:
df2 = pd.read_pickle('Finaldata.pkl')
df2.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_proportion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,133414,5428,19.0,18.0,0.947368,1.947368,5.736842,23.0,1.0,1.0,...,101,0.604790,7.260870,0.642670,12.0,7.0,6.0,0.500000,0.857143,0.666667
1,107612,49099,1.0,0.0,0.000000,13.000000,16.000000,3.0,0.0,0.0,...,53,0.304598,19.333333,0.306129,18.0,18.0,16.0,0.500000,0.277778,0.437500
2,69469,8021,1.0,0.0,0.000000,1.000000,4.000000,18.0,0.0,0.0,...,93,0.550296,6.035714,0.481418,20.0,3.0,19.0,0.800000,0.333333,0.736842
3,190574,6377,1.0,0.0,0.000000,2.000000,17.000000,11.0,0.0,0.0,...,76,0.410811,5.606061,0.465079,6.0,5.0,1.0,0.333333,0.800000,1.000000
4,196693,17758,3.0,2.0,0.666667,9.333333,24.666666,13.0,0.0,0.0,...,223,0.563131,24.750000,0.573914,22.0,18.0,28.0,0.636364,0.500000,0.821429
